# Table of Contents

1. [**Problem Description**](#1)
2. [**Gurobi**](#2)
3. [**Reference**](#3)

# 1- Problem Description <a id="1"></a>

<b> CSL is a chain of computer service stores. The number of hours of skilled repair time that CSL requires during the next five months is as follows:<br>
<ol><b>
    <li>Month 1 (January): 6000</li>
    <li>Month 2 (February): 7000</li>
    <li>Month 3 (March): 8000</li>
    <li>Month 4 (April): 9500</li>
    <li>Month 5 (May): 11000</li>
    </b>
</ol>
    
<b>At the beginning of January, 50 skilled technicians work for CSL. Each skilled technician can work up to 160 hours per month. To meet future demands, new technicians must be trained. It takes one month to train a new technician. During the month of training, a trainee must be supervised for 50 hours by an experienced technician. Each experienced technician is paid $2000 a month (even if he/she does not work the full 160 hours).
During the month of training , a trainee is paid 1000 dollars a month. At the end of each month, 5% of CSL's experienced technicians quit to join Plum Computers. Formulate an LP whos solution will enable CSL to minimize the labor cost incurred in meeting the service requirements for the next five months.</b>

## Mathematical Model

**sets**:<br>
**T**: Period of time = {'January', 'February', 'March', 'April', 'May'}<br>

**Parameter**:<br>
experienced technicians cost = 2000 dollars (EX_Tech_Cost)<br>
trainess cost = 1000 dollars (Trainess_Cost)<br>
Ex_Tech_Availabel = 160 h<br>
Trainee_requires = 50h<br>
Requires = [6000, 7000, 8000, 9500, 11000]<br>

**Variables**:<br>
$X_{t}$: *Number of technicians trained during month t<br>
$Y_{t}$: *Number of experienced technicians at the begining of the month t

## Model Formulation:

$$\begin{align}
\min Z= & TrainessCost*\sum_{t \in T} X_{t} + ExTechCost*\sum_{t \in T}Y_{t}\\
\text{S.t:}
&ExTechAvailabel*Y_{t} - TraineeRequires*X_{t} \ge Requires_{t} &\forall t \in T\\
& Y_{1} = 50\\
& Y_{t} = X_{t-1} + (1 - 0.05)*Y_{t-1} &\forall t \in T, t\ge2\\
&X_{t},  Y_{t} \ge 0 &\forall t \in T\\
\end{align}$$

# 2- Gurobi <a id="2"></a>

### Import packages

In [2]:
from gurobipy import GRB,Model
import numpy as np

### Parameters Definition

In [3]:
T = {'January', 'February', 'March', 'April', 'May'}

Variabels = [['X'+str(t) for t in range(len(T))], ['Y'+str(t) for t in range(len(T))]]

Parameters = {"Ex_Cost"  : 2000,
              "Tr_Cost"  : 1000,
              "Ex_Avail" : 160,
              "Tr_req"   : 50,
              "quit_per" : 0.05,
              "Requires" : np.array([6000, 7000, 8000, 9500, 11000])}

In [4]:
def output(m):  
    status_code = {1:'LOADED',
                   2: 'OPTIMAL',
                   3:'INFEASIBLE',
                   4:'INF_OR_UNBD',
                   5:'UNBOUNDED'}
    
    status = m.status
    print(f'The optimization status is: {status_code[status]}')
    
    if status == 2:
        print('Optimal solution:')
        for var in m.getVars():
            print(f'{var.varName} = {var.x}')
        print(f'Optimal Objective Function = {m.objVal}')

In [5]:
def Gurobi_Model(Var, Param):
    
    model = Model("Multiperiod Work Scheduling")
    
    model.setParam('OutputFlag', True)
    t = len(Var[0])
    E_C = Param["Ex_Cost"]
    T_C = Param["Tr_Cost"]
    E_A = Param["Ex_Avail"]
    T_R = Param["Tr_req"]
    Q_P = Param["quit_per"]
    Req = Param["Requires"]
    
    X = model.addVars(t, lb=0, ub=GRB.INFINITY, vtype = GRB.CONTINUOUS, name = Var[0])
    Y = model.addVars(t, lb=0, ub=GRB.INFINITY, vtype = GRB.CONTINUOUS, name = Var[1])
    
    model.setObjective(E_C*Y.sum("*") + T_C*X.sum("*"), GRB.MINIMIZE)
    
    model.addConstrs(E_A*Y[i] - T_R*X[i] >= Req[i] for i in range(t))
    model.addConstr (Y[0] == 50)
    model.addConstrs(Y[i] == (1-Q_P)*Y[i-1] + X[i-1] for i in range(t) if i>0)
    
    model.optimize()
    
    output(model)

In [6]:
Gurobi_Model(Variabels, Parameters)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-4210U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 10 rows, 10 columns and 23 nonzeros
Model fingerprint: 0xeb36277c
Coefficient statistics:
  Matrix range     [9e-01, 2e+02]
  Objective range  [1e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+04]
Presolve removed 6 rows and 5 columns
Presolve time: 0.03s
Presolved: 4 rows, 5 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.3815789e+32   9.847862e+29   1.381579e+02      0s
       3    5.9377651e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.05 seconds (0.00 work units)
Optimal objective  5.937765097e+05
The optimization status is: OPTIMAL
Optimal solution:
X0 = 0.0
X1 = 8.45

# 3- Reference <a id='3'></a>
[**Operations Research** Wayne L.Winston, page109](https://itslearningakarmazyan.files.wordpress.com/2015/09/operation-research-aplications-and-algorithms.pdf)